# MEA-Flow Tutorial: Basic Analysis Workflow

This notebook demonstrates the basic MEA-Flow workflow for analyzing multi-electrode array data.

## Overview

MEA-Flow provides tools for:
1. **Data Loading**: Support for MEA data formats
2. **Metrics Calculation**: Activity and synchrony measures
3. **Visualization**: Raster plots and comparisons
4. **Analysis**: Cross-condition comparisons

In [ ]:
# Standard libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

# MEA-Flow imports
from mea_flow import SpikeList, MEAMetrics
from mea_flow.analysis.metrics import AnalysisConfig

# Set up matplotlib
plt.style.use("default")
sns.set_palette("husl")

print("MEA-Flow Tutorial - Ready to begin!")

## 1. Data Loading and Creation

We'll create synthetic MEA data to demonstrate the workflow.

In [ ]:
# Create synthetic MEA data for demonstration
def create_synthetic_mea_data(condition_name="control", n_channels=16, duration=60.0):
    """Create synthetic MEA data for demonstration."""
    np.random.seed(42 if condition_name == "control" else 123 if condition_name == "treatment1" else 456)
    
    # Different activity levels for different conditions
    if condition_name == "control":
        base_rate = 2.0  # Hz
    elif condition_name == "treatment1":
        base_rate = 3.5  # Higher activity
    else:  # treatment2
        base_rate = 1.5  # Lower activity
    
    spike_data = {}
    
    for ch in range(n_channels):
        # Generate spikes for this channel - ensure minimum activity
        channel_rate = base_rate * np.random.uniform(0.8, 1.5)
        n_spikes = max(10, int(np.random.poisson(channel_rate * duration)))  # Ensure min 10 spikes
        spike_times = np.sort(np.random.uniform(0, duration, n_spikes))
        spike_data[ch] = spike_times
    
    return spike_data

# Create synthetic data for three experimental conditions
conditions = ["control", "treatment1", "treatment2"]
spike_lists = {}

for condition in conditions:
    print(f"Creating synthetic data for condition: {condition}")
    
    # Generate synthetic spike data
    spike_data = create_synthetic_mea_data(
        condition_name=condition,
        n_channels=16,
        duration=60.0
    )
    
    # Create SpikeList object
    spike_list = SpikeList(
        spike_data=spike_data,
        recording_length=60.0
    )
    
    spike_lists[condition] = spike_list
    
    print(f"  - Channels: {len(spike_list.channel_ids)}")
    print(f"  - Active channels: {len(spike_list.get_active_channels())}")
    print(f"  - Total spikes: {sum(train.n_spikes for train in spike_list.spike_trains.values())}")
    print(f"  - Recording length: {spike_list.recording_length:.1f} s")
    print()

print("Data loading completed!")

## 2. Basic Visualization

Create raster plots to visualize spike activity patterns.

In [ ]:
# Create raster plots for each condition
for condition, spike_list in spike_lists.items():
    print(f"Creating raster plot for {condition}...")
    
    # Plot first 20 seconds of activity
    fig, ax = plt.subplots(figsize=(12, 6))
    
    # Simple raster plot
    colors = plt.cm.tab10(np.linspace(0, 1, 16))
    for i, ch_id in enumerate(spike_list.get_active_channels()[:8]):  # Show first 8 channels
        spike_times = spike_list.spike_trains[ch_id].spike_times
        spike_times = spike_times[spike_times <= 20]  # First 20 seconds
        ax.scatter(spike_times, [ch_id] * len(spike_times), s=2, alpha=0.8, color=colors[i])
    
    ax.set_xlabel("Time (s)")
    ax.set_ylabel("Channel ID")
    ax.set_title(f"Raster Plot - {condition.title()}")
    ax.set_xlim(0, 20)
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    # Display summary statistics
    active_channels = spike_list.get_active_channels()
    firing_rates = [spike_list.spike_trains[ch].firing_rate for ch in active_channels]
    total_spikes = sum(spike_list.spike_trains[ch].n_spikes for ch in active_channels)
    
    print(f"\nSummary for {condition}:")
    print(f"  - Active channels: {len(active_channels)}/{len(spike_list.channel_ids)}")
    print(f"  - Mean firing rate: {np.mean(firing_rates):.2f} ± {np.std(firing_rates):.2f} Hz")
    print(f"  - Total spikes: {total_spikes}")
    print()

## 3. Metrics Analysis

Compute comprehensive metrics for each condition.

In [ ]:
# Compute metrics for all conditions
print("Computing comprehensive metrics for all conditions...")

# Initialize metrics analyzer
config = AnalysisConfig(
    time_bin_size=1.0,
    min_spikes_for_rate=5,
    burst_detection=False,  # Disable for simplicity
    network_burst_detection=False,
    n_pairs_sync=50  # Reduce for faster computation
)

metrics_analyzer = MEAMetrics(config=config)

# Compute all metrics for each condition
all_metrics = []

for condition, spike_list in spike_lists.items():
    print(f"Computing metrics for {condition}...")
    
    # Compute all metrics
    metrics_df = metrics_analyzer.compute_all_metrics(spike_list, grouping="global")
    
    # Add condition info
    metrics_df["condition"] = condition
    all_metrics.append(metrics_df)

# Combine all metrics
all_metrics_df = pd.concat(all_metrics, ignore_index=True)

print(f"\nMetrics computed for {len(all_metrics_df)} condition(s)")

# Display key metrics
key_metrics = [
    "condition", "activity_mean_firing_rate", "activity_active_channels_count",
    "activity_network_firing_rate", "regularity_cv_isi_mean", "synchrony_pearson_cc_mean"
]
available_metrics = [col for col in key_metrics if col in all_metrics_df.columns]

print("\n=== Key Metrics Summary ===")
print(all_metrics_df[available_metrics].round(4))

## 4. Metrics Comparison

Visualize differences between conditions.

In [ ]:
# Create comprehensive metrics comparison plots
print("Creating metrics comparison plots...")

# Create subplots for multiple metrics
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
axes = axes.flatten()

conditions = all_metrics_df["condition"].values
colors = ["skyblue", "orange", "lightgreen"]

# Plot 1: Mean Firing Rate
if "activity_mean_firing_rate" in all_metrics_df.columns:
    firing_rates = all_metrics_df["activity_mean_firing_rate"].values
    bars1 = axes[0].bar(conditions, firing_rates, alpha=0.7, color=colors)
    axes[0].set_ylabel("Mean Firing Rate (Hz)")
    axes[0].set_title("Mean Firing Rate by Condition")
    axes[0].grid(True, alpha=0.3)
    for bar, rate in zip(bars1, firing_rates):
        if not np.isnan(rate):
            axes[0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02,
                        f"{rate:.2f}", ha="center", va="bottom")

# Plot 2: Active Channels
if "activity_active_channels_count" in all_metrics_df.columns:
    active_channels = all_metrics_df["activity_active_channels_count"].values
    bars2 = axes[1].bar(conditions, active_channels, alpha=0.7, color=colors)
    axes[1].set_ylabel("Active Channels Count")
    axes[1].set_title("Active Channels by Condition")
    axes[1].grid(True, alpha=0.3)
    for bar, count in zip(bars2, active_channels):
        if not np.isnan(count):
            axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1,
                        f"{int(count)}", ha="center", va="bottom")

# Plot 3: CV ISI (Regularity)
if "regularity_cv_isi_mean" in all_metrics_df.columns:
    cv_isi = all_metrics_df["regularity_cv_isi_mean"].values
    bars3 = axes[2].bar(conditions, cv_isi, alpha=0.7, color=colors)
    axes[2].set_ylabel("CV ISI (Irregularity)")
    axes[2].set_title("Spike Regularity by Condition")
    axes[2].grid(True, alpha=0.3)
    for bar, cv in zip(bars3, cv_isi):
        if not np.isnan(cv):
            axes[2].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02,
                        f"{cv:.2f}", ha="center", va="bottom")

# Plot 4: Synchrony
if "synchrony_pearson_cc_mean" in all_metrics_df.columns:
    synchrony = all_metrics_df["synchrony_pearson_cc_mean"].values
    bars4 = axes[3].bar(conditions, synchrony, alpha=0.7, color=colors)
    axes[3].set_ylabel("Mean Correlation")
    axes[3].set_title("Network Synchrony by Condition")
    axes[3].grid(True, alpha=0.3)
    for bar, sync in zip(bars4, synchrony):
        if not np.isnan(sync):
            axes[3].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.002,
                        f"{sync:.3f}", ha="center", va="bottom")

# Rotate x-axis labels
for ax in axes:
    ax.tick_params(axis="x", rotation=45)

plt.tight_layout()
plt.show()

# Print numerical comparison
print("\n=== Detailed Numerical Comparison ===")
for i, condition in enumerate(conditions):
    print(f"\n{condition.upper()}:")
    if "activity_mean_firing_rate" in all_metrics_df.columns:
        print(f"  - Firing rate: {all_metrics_df.iloc[i]["activity_mean_firing_rate"]:.2f} Hz")
    if "activity_active_channels_count" in all_metrics_df.columns:
        print(f"  - Active channels: {int(all_metrics_df.iloc[i]["activity_active_channels_count"])}")
    if "regularity_cv_isi_mean" in all_metrics_df.columns:
        cv_val = all_metrics_df.iloc[i]["regularity_cv_isi_mean"]
        print(f"  - Spike regularity (CV ISI): {cv_val:.3f}" if not np.isnan(cv_val) else "  - Spike regularity: N/A")
    if "synchrony_pearson_cc_mean" in all_metrics_df.columns:
        sync_val = all_metrics_df.iloc[i]["synchrony_pearson_cc_mean"]
        print(f"  - Network synchrony: {sync_val:.3f}" if not np.isnan(sync_val) else "  - Network synchrony: N/A")

## Summary

This tutorial demonstrated the basic MEA-Flow workflow:

1. **Data Loading**: Created synthetic MEA data for multiple conditions
2. **Visualization**: Generated raster plots to visualize spike activity
3. **Metrics Analysis**: Computed comprehensive metrics (activity, regularity, synchrony)
4. **Comparison**: Compared metrics across experimental conditions

### Key Results:
- Successfully loaded and processed multi-condition MEA data
- Computed comprehensive activity, regularity, and synchrony metrics
- Visualized clear differences between treatment conditions
- Demonstrated the full MEA-Flow analysis pipeline

### Next Steps:
- Load real .spk or .mat files using 
- Explore burst detection and network burst analysis
- Perform manifold analysis for population dynamics
- Use statistical tests for rigorous condition comparisons
- Analyze time-resolved dynamics

For more advanced analysis, explore the other example scripts and notebooks in the MEA-Flow package.